In [229]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [230]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import pandas as pd
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df_train = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
df_test =  pd.read_csv("/kaggle/input/restaurant-revenue-prediction/test.csv.zip")

In [231]:
df_train = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
#to display first 5 rows
df_train.head(5)

In [232]:
#display basic structure of train data
df_train

In [233]:
#Total no of rows and columns
df_train.shape

In [234]:
#Displaying Column names
df_train.columns

In [235]:
df_train.info

In [236]:
#Checking the data type
df_train.dtypes

In [237]:
#Structure of testing data
df_test.shape

In [238]:
#count no of rows 
df_train.count()

In [239]:
#numerical columns
num_col=df_train.select_dtypes(exclude=['object']).columns
num_col

In [240]:
#categorical columns
cat_col=df_train.select_dtypes(include=['object']).columns
cat_col

In [241]:
#getting null values if any
df_train.columns[df_train.isna().any()].tolist()

In [242]:
df_train['Open Date'] = pd.to_datetime(df_train['Open Date'], format='%m/%d/%Y')   
df_train['OpenDays']=""

dateLastTrain = pd.DataFrame({'Date':np.repeat(['01/01/2018'],[len(df_train)]) })
dateLastTrain['Date'] = pd.to_datetime(dateLastTrain['Date'], format='%m/%d/%Y') 
dateLastTrain.head()


df_train['OpenDays'] = dateLastTrain['Date'] - df_train['Open Date']
df_train['OpenDays'] = df_train['OpenDays'].astype('timedelta64[D]').astype(int)

In [243]:
df_train.head(5)

In [244]:
cityPerc=df_train[["City Group","revenue"]].groupby(['City Group'],as_index=False).mean()
sns.barplot(x='City Group', y='revenue',data=cityPerc)

In [245]:
cityPerc=df_train[["City","revenue"]].groupby(['City'],as_index=False).mean()
newDF=cityPerc.sort_values(["revenue"],ascending=False)
sns.barplot(x='City', y='revenue',data=newDF.head(6))

In [246]:
cityPerc=df_train[["City","revenue"]].groupby(['City'],as_index=False).mean()
newDF=cityPerc.sort_values(["revenue"],ascending=True)
sns.barplot(x='City', y='revenue',data=newDF.head(6))

In [247]:
cityPerc=df_train[["Type","revenue"]].groupby(['Type'],as_index=False).mean()
sns.barplot(x='Type', y='revenue',data=cityPerc)

In [248]:
df_train.head()

In [249]:
correlation = df_train[num_col].corr()
f, ax=plt.subplots(figsize=(14,12))
plt.title('Correlation of numerical attributes',size=16)
sns.heatmap(correlation)
plt.show()

In [250]:
correlation['revenue'].sort_values(ascending=False)

In [251]:
df_train=df_train.drop('Open Date', axis=1)
df_train=df_train.drop('Id',axis=1)
df_train.head(5)

In [252]:
CityGroupDummyTrain=pd.get_dummies(df_train['City Group'])
df_train=df_train.join(CityGroupDummyTrain)
df_train=df_train.drop('City Group', axis=1)
df_train.head()

In [253]:
df_test['Open Date'] = pd.to_datetime(df_test['Open Date'], format='%m/%d/%Y')


df_test['OpenDays']=""

 
dateLastTest = pd.DataFrame({'Date':np.repeat(['01/01/2015'],[len(df_test)]) })
dateLastTest['Date'] = pd.to_datetime(dateLastTest['Date'], format='%m/%d/%Y')  


df_test['OpenDays'] = dateLastTest['Date'] - df_test['Open Date']


df_test['OpenDays'] = df_test['OpenDays'].astype('timedelta64[D]').astype(int)


df_test = df_test.drop('Open Date', axis=1)


citygroupDummyTest = pd.get_dummies(df_test['City Group'])
df_test = df_test.join(citygroupDummyTest)

df_test = df_test.drop('City Group', axis=1)

In [254]:
df_test.head()

In [255]:
import numpy
xTrain = pd.DataFrame({'OpenDays':df_train['OpenDays'].apply(numpy.log),
                      'Big Cities':df_train['Big Cities'], 'Other':df_train['Other'],
                      'P1':df_train['P1'], 'P2':df_train['P2'], 'P4':df_train['P4'],
                       'P5':df_train['P5'], 'P6':df_train['P6'],'P7':df_train['P7'], 
                       'P11':df_train['P11'], 'P14':df_train['P14'], 'P15':df_train['P15'],
                       'P17':df_train['P17'], 'P19':df_train['P19'], 'P20':df_train['P20'],
                       'P21':df_train['P21'], 'P23':df_train['P23'], 'P24':df_train['P24'],
                       'P25':df_train['P25'], 'P28':df_train['P28']})
                    
                      
yTrain = df_train['revenue'].apply(numpy.log)

xTest = pd.DataFrame({'OpenDays':df_test['OpenDays'].apply(numpy.log),
                      'Big Cities':df_test['Big Cities'], 'Other':df_test['Other'],
                      'P1':df_test['P1'], 'P2':df_test['P2'], 'P7':df_test['P7'],
                       'P4':df_test['P4'], 'P5':df_test['P5'], 'P6':df_test['P6'],
                        'P11':df_test['P11'], 'P14':df_test['P14'], 'P15':df_test['P15'],
                       'P17':df_test['P17'], 'P25':df_test['P25'],'P24':df_test['P24'],
                      'P19':df_test['P19'], 'P20':df_test['P20'], 'P21':df_test['P21'],
                     'P23':df_test['P23'],'P28':df_test['P28']})

In [256]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler(with_std  = True ,with_mean = True, copy = True)
xTrain = sc.fit_transform(xTrain)
xTest = sc.transform(xTest)

In [257]:
from sklearn import linear_model


# Create linear regression object
regr = linear_model.LinearRegression()


# Train the model using the training sets
regr.fit(xTrain, yTrain)

# Make predictions using the testing set
linear_predictions = regr.predict(xTest)

linear_predictions

In [258]:
regr.score(xTrain,yTrain)

In [259]:
forest = RandomForestRegressor(n_estimators=150)
forest.fit(xTrain, yTrain)


In [260]:
pred = forest.predict(xTest)
pred = numpy.exp(pred)
scoreOfModel=forest.score(xTrain, yTrain)
print("Score is calculated as: ",scoreOfModel)

In [261]:
pred = forest.predict(xTest)
pred = numpy.exp(pred)

In [262]:
pred

In [263]:
import xgboost as xgb
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
xgbr=best_xgb_model.fit(xTrain,yTrain)

In [264]:
xgbrscoreOfModel=xgbr.score(xTrain, yTrain)
print("Score is calculated as: ",xgbrscoreOfModel)

In [265]:
import numpy
xTrain = pd.DataFrame({'OpenDays':df_train['OpenDays'].apply(numpy.log),
                      'Big Cities':df_train['Big Cities'], 'Other':df_train['Other'],
                      'P1':df_train['P1'], 'P2':df_train['P2'], 'P3':df_train['P3'],
                       'P4':df_train['P4'], 'P5':df_train['P5'], 'P6':df_train['P6'],
                       'P7':df_train['P7'], 'P8':df_train['P8'], 'P9':df_train['P9'],
                       'P10':df_train['P10'], 'P11':df_train['P11'], 'P12':df_train['P12'],
                       'P13':df_train['P13'], 'P14':df_train['P14'], 'P15':df_train['P15'],
                       'P16':df_train['P16'], 'P17':df_train['P17'], 'P18':df_train['P18'],
                      'P19':df_train['P19'], 'P20':df_train['P20'], 'P21':df_train['P21'],
                      'P22':df_train['P22'], 'P23':df_train['P23'], 'P24':df_train['P24'],
                      'P25':df_train['P25'], 'P26':df_train['P26'], 'P27':df_train['P27'],
                      'P28':df_train['P28'], 'P29':df_train['P29'], 'P30':df_train['P30'],
                      'P31':df_train['P31'], 'P32':df_train['P32'], 'P33':df_train['P33'],
                      'P34':df_train['P34'], 'P35':df_train['P35'], 'P36':df_train['P36'],
                      'P37':df_train['P37']})

yTrain = df_train['revenue'].apply(numpy.log)

xTest = pd.DataFrame({'OpenDays':df_test['OpenDays'].apply(numpy.log),
                      'Big Cities':df_test['Big Cities'], 'Other':df_test['Other'],
                      'P1':df_test['P1'], 'P2':df_test['P2'], 'P3':df_test['P3'],
                       'P4':df_test['P4'], 'P5':df_test['P5'], 'P6':df_test['P6'],
                       'P7':df_test['P7'], 'P8':df_test['P8'], 'P9':df_test['P9'],
                       'P10':df_test['P10'], 'P11':df_test['P11'], 'P12':df_test['P12'],
                       'P13':df_test['P13'], 'P14':df_test['P14'], 'P15':df_test['P15'],
                       'P16':df_test['P16'], 'P17':df_test['P17'], 'P18':df_test['P18'],
                      'P19':df_test['P19'], 'P20':df_test['P20'], 'P21':df_test['P21'],
                      'P22':df_test['P22'], 'P23':df_test['P23'], 'P24':df_test['P24'],
                      'P25':df_test['P25'], 'P26':df_test['P26'], 'P27':df_test['P27'],
                      'P28':df_test['P28'], 'P29':df_test['P29'], 'P30':df_test['P30'],
                      'P31':df_test['P31'], 'P32':df_test['P32'], 'P33':df_test['P33'],
                      'P34':df_test['P34'], 'P35':df_test['P35'], 'P36':df_test['P36'],
                      'P37':df_test['P37']})

In [266]:
from sklearn import linear_model


# Create linear regression object
regr = linear_model.LinearRegression()


# Train the model using the training sets
regr.fit(xTrain, yTrain)

# Make predictions using the testing set
linear_predictions = regr.predict(xTest)

linear_predictions

In [267]:
regr.score(xTrain,yTrain)

In [268]:
forest = RandomForestRegressor(n_estimators=150)
forest.fit(xTrain, yTrain)

In [269]:
pred = forest.predict(xTest)
pred = numpy.exp(pred)
scoreOfModel=forest.score(xTrain, yTrain)
print("Score is calculated as: ",scoreOfModel)

In [270]:
import xgboost as xgb
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
xgbr=best_xgb_model.fit(xTrain,yTrain)

In [271]:
xgbrscoreOfModel=xgbr.score(xTrain, yTrain)
print("Score is calculated as: ",xgbrscoreOfModel)